### Importing the requried libraries

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import csv
import tensorflow.keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Input
from keras.models import Mode
from sklearn.model_selection import train_test_split
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.layers.embeddings import Embedding
from keras.layers import Dropout


### Mounting the Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Reading the data from csv file

In [ ]:
df = pd.read_csv('/content/drive/My Drive/finalcleandataTraining.csv')
#print(df.head())
xdata = df['message']
y_label = df[["toxic", "obscene", "insult", "racism", "sexism"]]
ydata = y_label.values

### Splitting the data into Training & Testing

In [ ]:
xtrain,xtest,y_train,ytest = train_test_split(xdata,ydata,test_size=0.2) 

### Creating the embedding dictionary

In [ ]:
maxlen = 64
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(xtrain)
vocab_size = len(tokenizer.word_index) + 1

embeddings_dictionary = dict()
glove_file = open('/content/drive/My Drive/Prj/glove.twitter.27B.200d.txt', encoding="utf8")
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:])
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

### Creating the embedding matrix from embedding dictionary

In [ ]:
embedding_matrix = zeros((vocab_size, 200))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

### Creating the model

In [ ]:
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 200, weights=[embedding_matrix], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(32, return_sequences=True)(embedding_layer)
LSTM_Layer_2 = LSTM(32, return_sequences=True)(LSTM_Layer_1)
Dropout_Layer = Dropout(0.5)(LSTM_Layer_2)
LSTM_Layer_3 = LSTM(32)(Dropout_Layer)
dense_layer_1 = Dense(5, activation='sigmoid')(LSTM_Layer_3)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

### Training the model

In [ ]:
X_train = tokenizer.texts_to_sequences(xtrain)
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
history = model.fit(X_train, y_train, batch_size=256, epochs=20, validation_split=0.2)

### Testing and evaluating the model

In [ ]:
X_test = tokenizer.texts_to_sequences(xtest)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
print(X_test)
print(model.metrics_names)
model.evaluate(X_test,ytest, batch_size=128)